In [1]:
from graphviz import Digraph


def trace(root): #this is a DFS to get all nodes and edges
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v.children:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format="png", graph_attr={"rankdir": "LR"})
    nodes, edges = trace(root)

    for n in nodes:
        uid = str(id(n))

        # value node (box)
        dot.node(uid, label=f"value={n.value:.4f}\ngrad={n.grad:.4f}", shape="record")

        # op node (small circle) if this node was produced by an op
        if n.op:
            op_id = uid + n.op
            dot.node(op_id, label=n.op, shape="circle")
            dot.edge(op_id, uid)  # op -> value

    # connect children -> op node (if present), else children -> value node
    for a, b in edges:
        b_uid = str(id(b))
        if b.op:
            dot.edge(str(id(a)), b_uid + b.op)  # child -> op
        else:
            dot.edge(str(id(a)), b_uid)         # leaf case

    return dot

class Node:
    def __init__(self, value, children=(), op=''):
        self.value = value
        self.grad = 0.0
        self._backward = lambda: None
        self.children = children
        self.op = op

    def __add__(self, other):   
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.value + other.value, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __sub__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.value - other.value, (self, other), '-')
        def _backward(): 
            self.grad += out.grad
            other.grad += -1*out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.value * other.value, (self, other), '*')    
        def _backward(): 
            self.grad += out.grad*other.value
            other.grad += out.grad*self.value
        out._backward = _backward
        return out
    
    def __repr__(self):
        return f"Node(value={self.value}, grad={self.grad}, op='{self.op}')"
    
    def relu(self):
        out = Node(0.0 if self.value < 0 else self.value, (self,), 'ReLU')

        def _backward():
            # d/dx ReLU(x) = 1 if x > 0 else 0
            self.grad += (out.value > 0) * out.grad

        out._backward = _backward
        return out
    
    
    def backward(self):
        visited = set()
        topo = []

        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v.children:
                    build(child)
                topo.append(v)

        build(self)

        self.grad = 1.0  # seed here

        for node in reversed(topo):
            node._backward()

    def zero_grad(self):
        nodes, _ = trace(self)
        for n in nodes:
            n.grad = 0.0


In [3]:
class Neuron:
    def __init__(self, nin, nonlin=True):
        self.weight = [Node(random.uniform(-1,1)) for _ in range(nin)]
        self.bias = Node(0.0)
        self.nonlin = nonlin

    def __call__(self, xss):
        xs = [xi if isinstance(xi,Node) else Node(xi) for xi in xss]
        out = self.bias
        for wi, xi in zip(self.weight, xs):
            out = out + xi*wi
        return out.relu() if self.nonlin else out

    def parameters(self):
        return self.weight + [self.bias]


In [4]:
class layers:
    def __init__(self, nin, nly, nonlin=True):
        self.neurons = [Neuron(nin, nonlin=nonlin) for _ in range(nly)]

    def __call__(self, xs):
        xs = [xi if isinstance(xi,Node) else Node(xi) for xi in xs]
        return [n(xs) for n in self.neurons]

    def parameters(self):
        params = []
        for n in self.neurons:
            params += n.parameters()
        return params


In [5]:
class MLP:
    def __init__(self, nin, layout):
        sz = [nin] + list(layout)
        self.layers = []
        for i in range(len(layout)):
            is_last = (i == len(layout)-1)
            self.layers.append(layers(sz[i], sz[i+1], nonlin=not is_last))

    def __call__(self, xs):
        for layer in self.layers:
            xs = layer(xs)
        return xs

    def parameters(self):
        params = []
        for layer in self.layers:
            params += layer.parameters()
        return params


In [6]:
import random
net = MLP(nin=4, layout=[8,4])

lr = 1e-4
for step in range(40000):
    x = [random.uniform(-5,5) for _ in range(4)]
    y = [2*xi for xi in x]

    y_pred = net(x)  # list of 4 Nodes

    loss = Node(0.0)
    for yp, yt in zip(y_pred, y):
        loss = loss + (yp - yt) * (yp - yt)

    # clear grads (do this on parameters, not loss unless you've implemented loss.zero_grad())
    for p in net.parameters():
        p.grad = 0.0

    loss.backward()

    for p in net.parameters():
        p.value -= lr * p.grad

    if step % 10000 == 0:
        print(step, loss.value)


0 212.9104133467926
10000 0.6381818944104549
20000 0.07113099323965341
30000 0.005147861361777542


In [7]:
test = [1.0,2.0,3.0,4.0]
pred = net(test)
print([p.value for p in pred])
test = [1.5,1.0,5.0,4.0]
pred = net(test)
print([p.value for p in pred])



[1.9969079789310487, 3.925072315383467, 5.967202304112263, 8.01480488153244]
[2.992788038724873, 1.923600184602718, 10.00151199752944, 8.009728413874726]
